In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

from vanilla_vae import VanillaVAE
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CelebA
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [ ]:
checkpoint_path = '../lightning_logs/version_0/checkpoints/epoch=49-step=56499.ckpt'
model = VanillaVAE.load_from_checkpoint(checkpoint_path, in_channels=3, latent_dim=128)

In [ ]:
SetRange = transforms.Lambda(lambda x: 2 * x - 1)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(148),
    transforms.Resize(64),
    transforms.ToTensor(),
    SetRange,
])

val_dataset = CelebA(root='../data', split='train', transform=transform, download=False)
val_loader = DataLoader(val_dataset, batch_size=144, num_workers=8, shuffle=True, drop_last=True)

In [ ]:
test_image, test_label = next(iter(val_loader))
test_image.shape

In [ ]:
plt.imshow(test_image[1].permute(1, 2, 0));

In [ ]:
result = model.reconstruct(test_image[1].unsqueeze(dim=0))
result = (result + 1) / 2.0
plt.imshow(result[0].permute(1, 2, 0).detach().numpy());

In [ ]:
img = make_grid(test_image.data, normalize=True, nrow=12)
plt.figure(figsize=(12, 12))
plt.axis('off')
plt.imshow(img.permute(1, 2, 0));

In [ ]:
recon_image = model.reconstruct(test_image)
img = make_grid(recon_image.data, normalize=True, nrow=12)
plt.figure(figsize=(12, 12))
plt.axis('off')
plt.imshow(img.permute(1, 2, 0));

In [ ]:
gen_image = model.sample(144)
gen_image.shape

In [ ]:
img = make_grid(gen_image.data, normalize=True, nrow=12)
plt.figure(figsize=(12, 12))
plt.axis('off')
plt.imshow(img.permute(1, 2, 0));